# Application: El Niño 3.4 visualisation

In [ ]:
import copernicus_marine_client as cmc
from datetime import datetime
import xarray as xr
import matplotlib.pyplot as plt
import s3fs
import os

## Fetch data from the 3.4 box (5N-5S, 170W-120W)

In [ ]:
start_datetime=datetime(2015,1,1)
end_datetime=datetime(2015,2,20)
minimal_longitude=-230.0
maximal_longitude=-50.0
minimal_latitude=-20.0
maximal_latitude=15.0
x_len=(maximal_longitude-minimal_longitude)/10
y_len=(maximal_latitude-minimal_latitude)/10

full_map=cmc.load_xarray_dataset(
    dataset_id="cmems_mod_glo_phy_my_0.083_P1M-m", 
    variables=["thetao"],
    minimal_longitude=minimal_longitude,
    maximal_longitude=maximal_longitude,
    minimal_latitude=minimal_latitude,
    maximal_latitude=maximal_latitude,
    minimal_depth=0,
    maximal_depth=100,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    force_service="geoseries"
)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(x_len, y_len))

full_map.thetao.isel(time=0, elevation=-1).plot(ax=ax, cmap='jet')

ax.add_patch(plt.Rectangle((360-170,-5.0), (360-120-(360-170)), (5.0-(-5.0)), fc=(1,0,0,0.0), ec=(0,0,0,1), lw=2))
ax.text(360-169,-4.6, 'Zone Niño 3.4')
plt.show()

## Fetch climato and reanalysis data on thetao for this box 

In [ ]:
query_result = cmc.describe(contains=["https://s3.waw2-1.cloudferro.com/swift/v1/AUTH_04a16d35c9e7451fa5894a700508c003/mdl-arco-geo/arco/GLOBAL_MULTIYEAR_PHY_001_030/cmems_mod_glo_phy_my_0.083-climatology_P1M-m_202112/geoChunked.zarr"], include_datasets=True)
query_result

In [ ]:
climato_EN_box34=cmc.load_xarray_dataset(
    dataset_url="https://s3.waw2-1.cloudferro.com/swift/v1/AUTH_04a16d35c9e7451fa5894a700508c003/mdl-arco-geo/arco/GLOBAL_MULTIYEAR_PHY_001_030/cmems_mod_glo_phy_my_0.083-climatology_P1M-m_202112/geoChunked.zarr", 
    variables=["thetao"],
    minimal_longitude=-170.0,
    maximal_longitude=-120.0,
    minimal_latitude=-5.0,
    maximal_latitude=5.0,
)

In [ ]:
query_result = cmc.describe(contains=["cmems_mod_glo_phy_my_0.083_P1D-m"], include_datasets=True)
query_result

In [ ]:
start_datetime=datetime(1993,1,1)
end_datetime=datetime(2020,12,31)

rea_EN_box34=cmc.load_xarray_dataset(
    dataset_id="cmems_mod_glo_phy_my_0.083_P1M-m", 
    variables=["thetao"],
    minimal_longitude=-170.0,
    maximal_longitude=-120.0,
    minimal_latitude=-5.0,
    maximal_latitude=5.0,
    minimal_depth=0, 
    maximal_depth=0.5,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    force_service="geoseries"
)

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=4, figsize=(12,12))

climato_EN_box34.isel(elevation=-1, time=0).thetao.plot(ax=axs[0][0], cmap='jet', vmin=23, vmax=30)
rea_EN_box34.isel(elevation=-1, time=0).thetao.plot(ax=axs[0][1], cmap='jet', vmin=23, vmax=30)
climato_EN_box34.isel(elevation=-1, time=3).thetao.plot(ax=axs[1][0], cmap='jet', vmin=23, vmax=30)
rea_EN_box34.isel(elevation=-1, time=3).thetao.plot(ax=axs[1][1], cmap='jet', vmin=23, vmax=30)
climato_EN_box34.isel(elevation=-1, time=6).thetao.plot(ax=axs[2][0], cmap='jet', vmin=23, vmax=30)
rea_EN_box34.isel(elevation=-1, time=6).thetao.plot(ax=axs[2][1], cmap='jet', vmin=23, vmax=30)
climato_EN_box34.isel(elevation=-1, time=9).thetao.plot(ax=axs[3][0], cmap='jet', vmin=23, vmax=30)
rea_EN_box34.isel(elevation=-1, time=9).thetao.plot(ax=axs[3][1], cmap='jet', vmin=23, vmax=30)
plt.tight_layout()
plt.draw()

In [ ]:
rea_EN_box=rea_EN_box34.groupby(rea_EN_box34.time.dt.month)
climato_EN_box=climato_EN_box34.groupby(climato_EN_box34.time.dt.month).mean()
diff_EN_box=rea_EN_box-climato_EN_box

In [ ]:
diff_EN_box

In [ ]:
diff_EN_box34=diff_EN_box

In [ ]:
fig, axs = plt.subplots(ncols=1, nrows=4, figsize=(6,12))

diff_EN_box34.isel(elevation=-1, time=0).thetao.plot(ax=axs[0], cmap='bwr', vmin=-4.0, vmax=4.0)
diff_EN_box34.isel(elevation=-1, time=3).thetao.plot(ax=axs[1], cmap='bwr', vmin=-4.0, vmax=4.0)
diff_EN_box34.isel(elevation=-1, time=6).thetao.plot(ax=axs[2], cmap='bwr', vmin=-4.0, vmax=4.0)
diff_EN_box34.isel(elevation=-1, time=9).thetao.plot(ax=axs[3], cmap='bwr', vmin=-4.0, vmax=4.0)
plt.tight_layout()
plt.draw()

In [ ]:
# Create filesystem object
S3_ENDPOINT_URL = os.environ["S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})

BUCKET_NAME="oidc-jsouchard"
fs.ls(BUCKET_NAME)

In [ ]:
FILE_PATH='oidc-jsouchard/meiv2.nc'

#To download the file on temporary workspace
#fs.download(FILE_PATH, 'meiv2.nc')
#To open from S3 directly
meiv2=xr.open_dataset(fs.open(FILE_PATH, mode="rb"))

In [ ]:
meiv2_rw=meiv2.rename({'MEI': 'thetao'}).sel(time=slice("1993-01-01", "2020-12-31"))
diff_rw=diff_EN_box34.mean(("longitude", "latitude")).isel(elevation=-1).drop_vars(['elevation', 'month'])
meiv2_rw=meiv2_rw.assign_coords(time=diff_rw.time)
corr=xr.corr(meiv2_rw.thetao, diff_rw.thetao)

In [ ]:
meiv2_rw.thetao.plot(label="Multivariate ENSO Index")
diff_rw.thetao.plot(label="Diff model - climato [°C]")
plt.title(f"R_corr= {corr:.3f}")
plt.legend()